# Capstone 1 (from Chapter 5) — Monte Carlo Estimation of Pi

In [1]:
!uv pip install numpy

Audited 1 package in 0.58ms


## Build Tools

In [2]:
from llm_agents_from_scratch.tools import PydanticFunctionTool

### Tool: `generate_random_sample`

In [40]:
import uuid

import numpy as np
from pydantic import BaseModel, Field

# Global registry to store samples
SAMPLE_REGISTRY: dict[str, list[tuple[float, float]]] = {}


class RandomSampleParams(BaseModel):
    """Params for generate_random_sample."""

    n: int = Field(description="The number of random points to generate")


class RandomSampleResult(BaseModel):
    """Result from generate_random_sample."""

    sample_id: str = Field(
        description="Pass this sample_id to monte_carlo_estimate",
    )
    n: int = Field(description="Number of points generated")


def generate_random_sample(params: RandomSampleParams) -> RandomSampleResult:
    """Generate n random points in [-1, 1] × [-1, 1].

    Returns a sample_id. Pass this sample_id directly to monte_carlo_estimate.
    """
    orig_pts = np.random.uniform(size=(params.n, 2))
    transformed = 1 - 2 * (1 - orig_pts)

    sample_id = str(uuid.uuid4())
    SAMPLE_REGISTRY[sample_id] = [tuple(pt) for pt in transformed.tolist()]

    return RandomSampleResult(sample_id=sample_id, n=params.n)


class MonteCarloEstimateParams(BaseModel):
    """Params for monte_carlo_estimate."""

    sample_id: str = Field(
        description="The sample_id returned by generate_random_sample",
    )


def monte_carlo_estimate(params: MonteCarloEstimateParams) -> float:
    """Estimate pi using Monte Carlo method.

    Args:
        params: Contains sample_id from generate_random_sample.

    Returns:
        Estimate of pi (float).
    """
    points = SAMPLE_REGISTRY[params.sample_id]
    n = len(points)
    inside = sum((x**2 + y**2) < 1 for x, y in points)
    return (inside / n) * 4


random_sample_tool = PydanticFunctionTool(generate_random_sample)
monte_carlo_estimate_tool = PydanticFunctionTool(monte_carlo_estimate)

In [44]:
rs = generate_random_sample(RandomSampleParams(n=1000))
str(rs)

"sample_id='d093f49e-76d2-4737-8f93-0f1d87f5f754' n=1000"

In [63]:
pi_estimate = monte_carlo_estimate(
    MonteCarloEstimateParams(sample_id="55165cbe-3cd1-4d68-b478-7f1336dfc18f"),
)
pi_estimate

2.984

In [46]:
monte_carlo_estimate_tool.parameters_json_schema

{'description': 'Params for monte_carlo_estimate.',
 'properties': {'sample_id': {'description': 'The sample_id returned by generate_random_sample',
   'title': 'Sample Id',
   'type': 'string'}},
 'required': ['sample_id'],
 'title': 'MonteCarloEstimateParams',
 'type': 'object'}

## Define our LLMAgent

In [47]:
from llm_agents_from_scratch import LLMAgent
from llm_agents_from_scratch.llms import OllamaLLM

llm = OllamaLLM(model="qwen2.5:3b")
llm_agent = LLMAgent(
    llm=llm,
    tools=[random_sample_tool, monte_carlo_estimate_tool],
)

## Define the Task

In [48]:
from llm_agents_from_scratch.data_structures import Task

In [66]:
instruction_template = """
You are given tools to estimate pi using Monte Carlo methods.

Your target: Estimate pi accurate to 4 decimal places (3.1415).

<tools>
1. `generate_random_sample(n)` → Returns a sample_id and n
2. `monte_carlo_estimate(sample_id)` → Returns a pi estimate (float)
</tools>

<workflow>
1. Call generate_random_sample(n=1000)
2. STOP. WAIT. Do not proceed until you receive the tool response.
3. Call monte_carlo_estimate with the sample_id you received
4. STOP. WAIT. Do not proceed until you receive the tool response.
5. Check if round(estimate, 4) == 3.1415
6. Repeat until converged for 10 consecutive estimates
</workflow>

<example>
User turn 1:
Call: generate_random_sample(n=1000)
[END YOUR RESPONSE HERE AND WAIT]

Tool response 1:
sample_id='abc-123' n=1000

User turn 2:
Call: monte_carlo_estimate(sample_id="abc-123")
[END YOUR RESPONSE HERE AND WAIT]

Tool response 2:
3.156

User turn 3:
Estimate: 3.156, rounded: 3.156 ≠ 3.1415
Call: generate_random_sample(n=1000)
[END YOUR RESPONSE HERE AND WAIT]
</example>

<critical>
YOU MUST MAKE EXACTLY ONE TOOL CALL PER RESPONSE.
After making a tool call, STOP IMMEDIATELY.
Do NOT anticipate the result.
Do NOT make a second tool call in the same response.
WAIT for the system to return the tool result before your next action.
</critical>

<warnings>
NEVER fabricate tool results.
NEVER make multiple tool calls in one response.
NEVER continue after a tool call - end your response immediately.
ALWAYS wait for the actual tool response before proceeding.
</warnings>
""".strip()

In [67]:
task = Task(
    instruction=instruction_template,
)

## Perform the Task

In [68]:
LOGGING_ENABLED = True

In [69]:
import logging

from llm_agents_from_scratch.logger import enable_console_logging

if LOGGING_ENABLED:
    enable_console_logging(logging.INFO)

In [70]:
handler = llm_agent.run(task, max_steps=5)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are given tools to estimate pi using Monte Carlo methods.

Your target: Estimate pi accurate to 4 decimal places (3.1415).

<tool...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are given tools to estimate pi using Monte Carlo methods.

Your target: Estimate pi accurate to 4 decimal places (3.1415).

<t...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ✅ Step Result: Call: generate_random_sample(n=1000)
INFO (llm_agents_fs.TaskHandler) :      🧠 New Step: Call: generate_random_sample(n=1000)
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: Call: generate_random_sample(n=1000)
INFO (llm_agents_fs.TaskHandler) :      🛠️ Executing Tool Call: generate_random_sample
INFO (llm_agents_fs.TaskHandler) :      ✅ Successful Tool Call: sample_id='398e57dc-657a-4325-8ce5-d50a6cbddeee' n=1000
INFO (llm_agents_fs.TaskHandler) :      ✅ Step Result: Alright, we have a sample ID. We will now move on to 

In [71]:
handler.result()

TaskResult(task_id='e5264299-e34f-4a8c-a870-15b24202a478', content='Estimate: 3.1419, rounded: 3.1419 ≠ 3.1415')

In [72]:
print(handler.rollout)

=== Task Step Start ===

💬 assistant: The current instruction is 'You are given tools to estimate pi using Monte Carlo methods.

Your target: Estimate pi accurate to 4 decimal places (3.1415).

<tools>
1. `generate_random_sample(n)` → Returns a sample_id and n
2. `monte_carlo_estimate(sample_id)` → Returns a pi estimate (float)
</tools>

<workflow>
1. Call generate_random_sample(n=1000)
2. STOP. WAIT. Do not proceed until you receive the tool response.
3. Call monte_carlo_estimate with the sample_id you received
4. STOP. WAIT. Do not proceed until you receive the tool response.
5. Check if round(estimate, 4) == 3.1415
6. Repeat until converged for 10 consecutive estimates
</workflow>

<example>
User turn 1:
Call: generate_random_sample(n=1000)
[END YOUR RESPONSE HERE AND WAIT]

Tool response 1:
sample_id='abc-123' n=1000

User turn 2:
Call: monte_carlo_estimate(sample_id="abc-123")
[END YOUR RESPONSE HERE AND WAIT]

Tool response 2:
3.156

User turn 3:
Estimate: 3.156, rounded: 3.156 ≠

In [32]:
handler.result()

MaxStepsReachedError: Max steps reached.